##### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from matplotlib import style 
import re
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.tree import DecisionTreeRegressor

In [2]:
warnings.filterwarnings('ignore')
style.use("seaborn")
pd.set_option("display.max_columns", None)
pd.set_option("max_rows", None)

In [3]:
data = pd.read_csv("clean_sales_data.csv", index_col = 0)

In [4]:
data.head()

,date,ay,ceyrek,mevsim,mevsim_2,mevsim_3,yil,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz
1,2012-01-01,1,1,1,1,1,2012,495093.0,49,16,24,1,7,9,33,10.61,79.029069,0,0,2.383768,-0.033665,1.847814,-0.010829,60.5,80.2,93.35,98.09,102.25,81.20,92.08,83.27,93.19,77.05,14.05,88.65,80.77,78.60,91.85,0,-18.5,-18.9,-3.3,-25.9,-19.5,-27.1,-22.4,4.4,-14.8,-15.7,-6.3,-24.6,-18.2,-28.1,-12.3,3.3,91.4,92.1,104.5,89.9,89.0,84.2,89.9,107.7,48.5,77793.0,105.92
2,2012-02-01,2,1,1,1,1,2012,471891.0,56,20,29,1,8,10,32,10.43,79.648792,0,0,2.325767,-0.024939,1.759533,-0.050173,61.9,79.7,95.35,99.96,103.49,81.64,92.47,84.82,93.60,80.08,11.94,93.98,81.01,79.52,96.06,0,-17.7,-18.1,-3.6,-25.5,-17.3,-25.6,-22.5,6.8,-13.0,-14.1,-7.3,-22.2,-15.3,-23.7,-7.3,3.4,92.8,93.1,105.1,90.0,91.8,83.8,89.8,109.1,48.5,77793.0,109.87
3,2012-03-01,3,1,2,1,2,2012,489523.0,46,20,27,1,7,9,30,10.43,79.336559,0,0,2.361768,0.015243,1.787882,0.015856,75.2,80.7,95.34,100.56,102.92,78.99,91.89,83.14,98.98,77.35,11.69,91.23,80.63,79.23,94.88,0,-16.7,-17.1,-3.5,-28.8,-13.4,-23.1,-24.2,7.9,-11.5,-13.1,-5.8,-22.2,-16.9,-21.1,-15.1,4.0,93.1,93.5,104.4,89.3,93.6,87.8,87.4,110.9,48.5,77793.0,108.30
4,2012-04-01,4,2,2,2,2,2012,474710.0,48,20,23,1,7,9,26,11.14,75.748829,0,0,2.354530,-0.003074,1.788420,0.000301,80.0,85.9,92.36,94.29,94.76,74.60,87.92,80.35,98.68,80.67,12.73,80.31,73.53,75.35,94.07,0,-17.9,-18.4,-4.5,-31.8,-14.2,-29.4,-25.0,3.0,-10.7,-12.1,-8.1,-21.2,-9.4,-20.9,-2.5,4.4,91.7,91.6,103.6,86.4,93.1,80.7,91.9,106.9,48.5,77793.0,109.92
5,2012-05-01,5,2,2,2,2,2012,526222.0,43,24,26,1,8,9,29,8.28,78.109503,0,1,2.316350,-0.016483,1.805627,0.009530,89.2,86.1,92.79,99.40,102.17,79.23,89.54,82.75,102.14,85.31,12.47,85.58,79.99,77.82,94.79,0,-17.9,-18.3,-3.8,-33.5,-11.9,-30.0,-24.7,4.5,-16.3,-18.2,-14.2,-23.8,-20.3,-28.2,-6.8,5.3,89.0,89.2,101.6,86.6,90.8,75.8,86.6,108.0,48.5,77793.0,110.41


In [5]:
data.shape

(99, 66)

In [6]:
data = data.set_index("date")

### FEATURE ENGINEERING -- ADDING LAGS 

**I added last couple of months sales figures in order to increase models predicting capabilities.**

In [7]:
data["lag1"] = data["TV_Pazar"].shift(1)
data["lag2"] = data["TV_Pazar"].shift(2)
data["lag3"] = data["TV_Pazar"].shift(3)

In [8]:
data["mean_son_3_ay"] = ( data["lag1"] + data["lag2"] + data["lag3"] ) / 3

In [9]:
data.head(3)

,ay,ceyrek,mevsim,mevsim_2,mevsim_3,yil,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,1,1,1,2012,495093.0,49,16,24,1,7,9,33,10.61,79.029069,0,0,2.383768,-0.033665,1.847814,-0.010829,60.5,80.2,93.35,98.09,102.25,81.20,92.08,83.27,93.19,77.05,14.05,88.65,80.77,78.60,91.85,0,-18.5,-18.9,-3.3,-25.9,-19.5,-27.1,-22.4,4.4,-14.8,-15.7,-6.3,-24.6,-18.2,-28.1,-12.3,3.3,91.4,92.1,104.5,89.9,89.0,84.2,89.9,107.7,48.5,77793.0,105.92,NaN,NaN,NaN,NaN
2012-02-01,2,1,1,1,1,2012,471891.0,56,20,29,1,8,10,32,10.43,79.648792,0,0,2.325767,-0.024939,1.759533,-0.050173,61.9,79.7,95.35,99.96,103.49,81.64,92.47,84.82,93.60,80.08,11.94,93.98,81.01,79.52,96.06,0,-17.7,-18.1,-3.6,-25.5,-17.3,-25.6,-22.5,6.8,-13.0,-14.1,-7.3,-22.2,-15.3,-23.7,-7.3,3.4,92.8,93.1,105.1,90.0,91.8,83.8,89.8,109.1,48.5,77793.0,109.87,495093.0,NaN,NaN,NaN
2012-03-01,3,1,2,1,2,2012,489523.0,46,20,27,1,7,9,30,10.43,79.336559,0,0,2.361768,0.015243,1.787882,0.015856,75.2,80.7,95.34,100.56,102.92,78.99,91.89,83.14,98.98,77.35,11.69,91.23,80.63,79.23,94.88,0,-16.7,-17.1,-3.5,-28.8,-13.4,-23.1,-24.2,7.9,-11.5,-13.1,-5.8,-22.2,-16.9,-21.1,-15.1,4.0,93.1,93.5,104.4,89.3,93.6,87.8,87.4,110.9,48.5,77793.0,108.30,471891.0,495093.0,NaN,NaN


In [10]:
data["lag4"] = data["TV_Pazar"].shift(4)
data["lag5"] = data["TV_Pazar"].shift(5)
data["lag6"] = data["TV_Pazar"].shift(6)

In [11]:
data["mean_son_6_ay"] = ( data["lag1"] + data["lag2"] + data["lag3"] 
                        + data["lag4"] + data["lag5"] + data["lag6"] ) / 6

In [12]:
data["lag7"] = data["TV_Pazar"].shift(7)
data["lag8"] = data["TV_Pazar"].shift(8)
data["lag9"] = data["TV_Pazar"].shift(9)

In [13]:
data["mean_son_9_ay"] = ( data["lag1"] + data["lag2"] + data["lag3"] 
                        + data["lag4"] + data["lag5"] + data["lag6"] 
                        + data["lag7"] + data["lag8"] + data["lag9"] ) / 9

In [14]:
data["lag10"] = data["TV_Pazar"].shift(10)
data["lag11"] = data["TV_Pazar"].shift(11)
data["lag12"] = data["TV_Pazar"].shift(12)

In [15]:
data["mean_son_12_ay"] = ( data["lag1"] + data["lag2"] + data["lag3"] 
                        + data["lag4"] + data["lag5"] + data["lag6"] 
                        + data["lag7"] + data["lag8"] + data["lag9"]  
                        + data["lag10"] + data["lag11"] + data["lag12"]) / 12

In [16]:
data.head(13)

,ay,ceyrek,mevsim,mevsim_2,mevsim_3,yil,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,1,1,1,2012,495093.0,49,16,24,1,7,9,33,10.61,79.029069,0,0,2.383768,-0.033665,1.847814,-0.010829,60.5,80.2,93.35,98.09,102.25,81.20,92.08,83.27,93.19,77.05,14.05,88.65,80.77,78.60,91.85,0,-18.5,-18.9,-3.3,-25.9,-19.5,-27.1,-22.4,4.4,-14.8,-15.7,-6.3,-24.6,-18.2,-28.1,-12.3,3.3,91.4,92.1,104.5,89.9,89.0,84.2,89.9,107.7,48.5,77793.0,105.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-01,2,1,1,1,1,2012,471891.0,56,20,29,1,8,10,32,10.43,79.648792,0,0,2.325767,-0.024939,1.759533,-0.050173,61.9,79.7,95.35,99.96,103.49,81.64,92.47,84.82,93.60,80.08,11.94,93.98,81.01,79.52,96.06,0,-17.7,-18.1,-3.6,-25.5,-17.3,-25.6,-22.5,6.8,-13.0,-14.1,-7.3,-22.2,-15.3,-23.7,-7.3,3.4,92.8,93.1,105.1,90.0,91.8,83.8,89.8,109.1,48.5,77793.0,109.87,495093.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-01,3,1,2,1,2,2012,489523.0,46,20,27,1,7,9,30,10.43,79.336559,0,0,2.361768,0.015243,1.787882,0.015856,75.2,80.7,95.34,100.56,102.92,78.99,91.89,83.14,98.98,77.35,11.69,91.23,80.63,79.23,94.88,0,-16.7,-17.1,-3.5,-28.8,-13.4,-23.1,-24.2,7.9,-11.5,-13.1,-5.8,-22.2,-16.9,-21.1,-15.1,4.0,93.1,93.5,104.4,89.3,93.6,87.8,87.4,110.9,48.5,77793.0,108.30,471891.0,495093.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-01,4,2,2,2,2,2012,474710.0,48,20,23,1,7,9,26,11.14,75.748829,0,0,2.354530,-0.003074,1.788420,0.000301,80.0,85.9,92.36,94.29,94.76,74.60,87.92,80.35,98.68,80.67,12.73,80.31,73.53,75.35,94.07,0,-17.9,-18.4,-4.5,-31.8,-14.2,-29.4,-25.0,3.0,-10.7,-12.1,-8.1,-21.2,-9.4,-20.9,-2.5,4.4,91.7,91.6,103.6,86.4,93.1,80.7,91.9,106.9,48.5,77793.0,109.92,489523.0,471891.0,495093.0,485502.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-01,5,2,2,2,2,2012,526222.0,43,24,26,1,8,9,29,8.28,78.109503,0,1,2.316350,-0.016483,1.805627,0.009530,89.2,86.1,92.79,99.40,102.17,79.23,89.54,82.75,102.14,85.31,12.47,85.58,79.99,77.82,94.79,0,-17.9,-18.3,-3.8,-33.5,-11.9,-30.0,-24.7,4.5,-16.3,-18.2,-14.2,-23.8,-20.3,-28.2,-6.8,5.3,89.0,89.2,101.6,86.6,90.8,75.8,86.6,108.0,48.5,77793.0,110.41,474710.0,489523.0,471891.0,478708.000000,495093.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-01,6,2,3,2,3,2012,491472.0,38,29,21,1,19,10,35,8.87,76.758228,0,1,2.285619,-0.013445,1.824833,0.010525,93.1,85.4,94.38,96.43,97.99,78.30,90.39,81.79,101.25,81.62,11.82,90.51,80.51,76.66,95.05,0,-17.7,-18.0,-4.0,-30.0,-12.1,-30.5,-24.5,4.7,-12.8,-14.2,-14.6,-17.6,-14.2,-16.9,3.7,2.7,88.8,88.8,99.8,86.5,89.0,80.0,88.7,105.4,48.5,77793.0,110.11,526222.0,474710.0,489523.0,496818.333333,471891.0,495093.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-01,7,3,3,2,3,2012,478926.0,49,37,24,1,11,9,45,9.07,77.012661,0,0,2.232927,-0.023598,1.813591,-0.006199,90.9,87.2,95.09,98.16,99.41,78.35,90.84,82.06,101.52,75.24,10.70,93.38,80.83,76.70,94.95,0,-18.4,-19.1,-6.5,-33.9,-15.5,-28.9,-21.7,2.9,-14.1,-15.7,-13.7,-21.0,-15.0,-25.3,4.1,4.1,86.8,86.7,96.5,85.0,86.6,79.0,90.2,107.9,48.5,77793.0,111.01,491472.0,526222.0,4747

In [17]:
data2 = data.dropna()

In [18]:
data2.loc[:, "lag1" : "mean_son_12_ay"] = data2.loc[:, "lag1" : "mean_son_12_ay"].apply(round)

In [19]:
data2.head()

,ay,ceyrek,mevsim,mevsim_2,mevsim_3,yil,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,1,1,1,1,2013,571770.0,44,16,25,1,8,6,25,7.31,75.787368,0,0,2.352659,0.003470,1.772368,-0.008637,68.5,91.3,94.97,93.02,97.41,75.84,91.82,83.27,93.45,78.84,11.19,86.23,78.90,75.38,90.98,0,-19.7,-20.5,-6.5,-34.3,-19.2,-30.0,-20.1,1.4,-15.8,-17.6,-14.8,-20.3,-15.9,-31.1,11.3,4.9,86.9,87.2,98.7,85.1,85.6,79.4,93.7,102.7,48.5,79853.0,113.36,600658.0,588748.0,617001.0,602136.0,507079.0,470569.0,478926.0,543830.0,491472.0,526222.0,474710.0,528376.0,489523.0,471891.0,495093.0,517658.0
2013-02-01,2,1,1,1,1,2013,488879.0,46,17,32,1,9,7,33,7.03,76.660100,0,0,2.377090,0.010278,1.776035,0.002065,69.9,91.8,95.99,93.94,99.05,76.07,90.86,84.82,97.52,76.59,10.38,90.31,79.97,76.55,91.62,0,-19.5,-20.4,-5.6,-34.0,-18.9,-30.7,-21.7,0.4,-15.9,-18.1,-15.7,-16.6,-16.6,-31.1,7.4,7.8,87.7,87.7,99.3,87.1,85.8,78.7,92.3,105.4,48.5,79308.0,112.75,571770.0,600658.0,588748.0,587059.0,617001.0,507079.0,470569.0,559304.0,478926.0,491472.0,526222.0,539161.0,474710.0,489523.0,471891.0,524047.0
2013-03-01,3,1,2,1,2,2013,488910.0,43,19,29,1,8,7,28,7.29,74.911700,0,0,2.349043,-0.011940,1.810457,0.019013,84.3,92.4,97.36,93.62,98.05,75.42,90.79,83.14,97.78,81.24,11.90,88.30,79.83,74.83,85.94,0,-19.2,-20.2,-5.2,-32.4,-19.9,-30.9,-19.9,0.9,-17.3,-19.6,-18.6,-16.9,-17.8,-32.3,-2.6,9.0,87.7,87.7,99.8,85.6,85.2,81.6,93.9,108.7,48.5,77133.0,112.87,488879.0,571770.0,600658.0,553769.0,588748.0,617001.0,507079.0,562356.0,470569.0,478926.0,491472.0,535011.0,526222.0,474710.0,489523.0,525463.0
2013-04-01,4,2,2,2,2,2013,474507.0,48,21,28,1,8,8,27,6.13,75.620582,0,0,2.339733,-0.003979,1.799690,-0.005983,87.8,92.3,95.69,97.67,101.27,75.76,91.11,80.35,94.90,81.68,11.33,88.62,79.77,75.22,87.36,0,-18.6,-19.5,-5.9,-32.2,-19.8,-26.5,-19.5,3.9,-19.3,-21.5,-23.9,-13.6,-22.4,-30.9,-0.9,6.4,85.9,85.9,97.0,86.3,82.1,78.6,92.1,107.5,48.5,79236.0,113.85,488910.0,488879.0,571770.0,516520.0,600658.0,588748.0,617001.0,559328.0,507079.0,470569.0,478926.0,534727.0,491472.0,526222.0,474710.0,525412.0
2013-05-01,5,2,2,2,2,2013,468524.0,42,27,30,1,8,7,31,6.51,77.461800,0,0,2.370900,0.013146,1.826050,0.014435,97.9,92.8,99.34,101.12,104.74,75.50,94.43,82.75,95.95,85.71,14.79,89.36,79.94,77.17,85.41,0,-18.5,-19.3,-4.5,-32.8,-22.3,-26.4,-18.5,0.1,-16.6,-18.5,-16.0,-18.5,-23.3,-25.7,0.2,4.8,86.8,86.9,98.4,87.1,82.1,79.7,94.0,104.6,48.5,75980.0,112.25,474507.0,488910.0,488879.0,484099.0,571770.0,600658.0,588748.0,535579.0,617001.0,507079.0,470569.0,534236.0,478926.0,491472.0,526222.0,525395.0


In [20]:
data2.tail()

,ay,ceyrek,mevsim,mevsim_2,mevsim_3,yil,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-01,11,4,4,1,2,2019,546334.0,83,56,94,16,25,18,77,10.56,59.415789,0,0,6.351286,-0.007263,5.741548,-0.009303,87.4,82.4,95.50,64.93,78.51,71.93,79.34,71.78,46.53,56.54,12.98,77.94,40.88,59.90,58.82,0,-6.3,-7.2,-2.0,-10.3,-6.0,-16.1,-8.8,-19.3,0.9,-0.2,-5.1,-0.1,-6.0,9.8,-13.7,0.9,100.9,100.7,98.6,101.8,101.8,99.8,89.7,91.4,49.5,120341.0,77.40,463329.0,371905.0,328424.0,387886.0,346569.0,381932.0,392833.0,380832.0,349634.0,400725.0,333456.0,374312.0,342839.0,465824.0,448594.0,385505.0
2019-12-01,12,4,1,1,1,2019,504745.0,77,48,85,19,24,21,72,11.84,58.225631,0,0,6.492559,0.021759,5.848150,0.018228,114.9,87.3,94.82,65.71,76.52,71.81,77.69,73.88,48.29,56.49,12.51,74.63,40.66,58.77,57.09,0,-7.0,-8.1,-3.7,-12.1,-8.8,-14.2,-7.1,-18.8,1.8,0.7,-5.5,3.7,-3.2,12.0,-15.6,1.6,101.0,100.9,99.1,102.7,101.4,101.5,87.3,95.4,49.5,120287.0,76.09,546334.0,463329.0,371905.0,460523.0,328424.0,346569.0,381932.0,406416.0,392833.0,349634.0,400725.0,397965.0,333456.0,342839.0,465824.0,393650.0
2020-01-01,1,1,1,1,1,2020,432595.0,78,44,86,17,23,20,67,12.15,58.862529,0,0,6.586905,0.014323,5.928827,0.013608,62.3,83.5,95.81,64.57,77.38,68.88,79.50,72.80,43.96,56.00,11.38,71.54,42.70,58.82,56.80,0,-7.1,-8.1,-3.6,-11.5,-9.0,-13.2,-6.5,-19.5,1.2,-0.1,-6.2,1.6,-2.2,10.2,-21.8,4.4,102.5,102.6,101.2,101.5,103.9,101.3,90.7,97.0,51.3,121496.0,75.55,504745.0,546334.0,463329.0,504803.0,371905.0,328424.0,346569.0,426884.0,381932.0,392833.0,349634.0,409523.0,400725.0,333456.0,342839.0,396894.0
2020-02-01,2,1,1,1,1,2020,412781.0,73,48,91,19,24,23,66,12.37,57.381955,0,0,6.605785,0.002858,6.055370,0.020898,62.3,83.5,93.57,67.66,75.94,71.03,76.81,72.18,43.99,55.64,10.21,72.56,44.66,57.26,53.78,0,-5.9,-6.6,-2.6,-7.9,-7.3,-13.7,-6.2,-20.4,1.3,-0.2,-6.5,1.9,-5.9,10.3,-12.3,2.9,103.0,103.4,101.8,102.7,105.5,101.3,95.5,96.1,52.4,124076.0,75.13,432595.0,504745.0,546334.0,494558.0,463329.0,371905.0,328424.0,441222.0,346569.0,381932.0,392833.0,418741.0,349634.0,400725.0,333456.0,404373.0
2020-03-01,3,1,2,1,2,2020,383645.0,92,49,86,25,28,30,66,11.86,54.900000,0,0,7.000000,0.062715,6.330000,0.067665,62.3,83.5,94.85,68.30,75.57,70.57,78.58,75.47,40.47,54.61,10.70,73.76,42.06,58.20,57.63,0,-10.4,-11.6,-7.6,-11.6,-11.4,-21.2,-8.2,-19.9,-6.1,-8.3,-20.2,-1.4,-8.5,0.4,-12.9,2.1,94.8,94.5,92.0,99.3,100.6,83.7,92.0,90.7,48.1,120516.0,72.87,412781.0,432595.0,504745.0,450040.0,546334.0,463329.0,371905.0,455282.0,328424.0,346569.0,381932.0,420957.0,392833.0,349634.0,400725.0,410984.0


In [21]:
data2.shape

(87, 81)

In [22]:
X = data2

In [23]:
y = data2["TV_Pazar"]

In [24]:
X.shape

(87, 81)

In [25]:
y.shape

(87,)

In [26]:
X_train = X[:-12]

In [27]:
X_train.shape

(75, 81)

In [29]:
X_train.drop("yil", axis = 1, inplace = True)

In [30]:
X_holdout = X[-12:]

In [31]:
X_holdout.drop("yil", axis = 1, inplace = True)

In [32]:
X_holdout.shape, X_train.shape

((12, 80), (75, 80))

In [33]:
X_holdout.head()

,ay,ceyrek,mevsim,mevsim_2,mevsim_3,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,otv,futbol,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,secim,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-01,4,2,2,2,2,349634.0,59,45,73,38,31,41,65,19.50,64.453024,1,0,6.459143,0.044909,5.746105,0.051228,79.0,81.6,97.17,62.72,82.42,75.01,82.24,72.66,50.47,63.92,14.48,81.20,32.83,63.54,63.46,0,-6.5,-7.4,-0.7,-6.1,-11.1,-13.4,-11.1,-18.3,0.3,-1.1,-6.5,5.7,-4.1,4.5,5.1,-10.9,104.0,103.6,104.5,103.9,99.4,99.7,98.6,85.0,46.8,108311.0,72.73,400725.0,333456.0,342839.0,359007.0,465824.0,448594.0,382862.0,395717.0,379567.0,410789.0,456198.0,402317.0,459589.0,524049.0,457754.0,421854.0
2019-05-01,5,2,2,2,2,392833.0,59,64,77,38,33,37,69,18.71,55.507248,1,0,6.779509,0.047255,6.060359,0.051854,86.2,81.1,94.46,59.91,70.15,69.14,73.95,69.02,39.79,53.93,10.30,76.68,32.11,55.28,56.29,1,-5.8,-6.5,-1.4,-3.7,-9.5,-12.1,-8.3,-23.7,0.6,-0.9,-4.3,5.3,-6.3,5.8,3.0,-11.6,105.0,105.0,104.9,105.1,104.2,101.6,93.9,79.7,45.3,111580.0,69.94,349634.0,400725.0,333456.0,361272.0,342839.0,465824.0,448594.0,390179.0,382862.0,379567.0,410789.0,390477.0,456198.0,459589.0,524049.0,412844.0
2019-06-01,6,2,3,2,3,381932.0,62,100,94,29,33,27,82,15.72,57.742552,1,0,6.568718,-0.032090,5.822400,-0.040870,85.8,92.1,97.34,60.26,73.94,68.90,78.29,69.43,40.46,57.25,10.87,79.99,31.35,57.63,56.30,1,-6.2,-7.2,-3.1,-2.1,-9.3,-14.0,-10.5,-21.6,1.2,0.1,-3.1,6.3,-6.6,7.7,-7.5,-7.2,103.1,102.9,101.9,103.8,103.3,99.9,94.3,84.9,47.9,114985.0,72.32,392833.0,349634.0,400725.0,381064.0,333456.0,342839.0,465824.0,380885.0,448594.0,382862.0,379567.0,388482.0,410789.0,456198.0,459589.0,401909.0
2019-07-01,7,3,3,2,3,346569.0,67,87,88,31,36,32,80,16.65,56.725613,0,0,6.380609,-0.029481,5.682395,-0.024638,84.7,81.0,96.88,59.80,73.37,67.54,76.97,65.82,40.26,59.39,11.04,79.86,34.47,56.49,55.58,0,-5.7,-6.6,-2.9,-4.9,-7.3,-11.8,-6.9,-22.8,0.6,-0.6,-6.9,3.0,-7.1,11.9,-11.7,-10.9,102.4,102.3,99.2,105.3,102.8,101.6,93.6,81.3,46.7,116480.0,75.01,381932.0,392833.0,349634.0,374800.0,400725.0,333456.0,342839.0,366903.0,465824.0,448594.0,382862.0,388744.0,379567.0,410789.0,456198.0,395438.0
2019-08-01,8,3,3,2,3,328424.0,66,100,90,30,39,31,80,15.01,58.501034,0,0,6.267294,-0.018080,5.634906,-0.008428,77.4,80.7,94.88,61.68,74.27,69.76,77.81,69.59,38.72,60.84,12.47,79.19,36.29,58.25,58.27,0,-6.2,-7.1,-3.9,-6.2,-7.7,-12.9,-11.4,-22.7,1.6,0.6,-4.9,6.1,-7.6,12.9,-29.0,-5.4,102.7,102.6,99.6,107.8,103.1,100.5,91.9,87.1,48.0,117118.0,76.94,346569.0,381932.0,392833.0,373778.0,349634.0,400725.0,333456.0,367525.0,342839.0,465824.0,448594.0,384712.0,382862.0,379567.0,410789.0,386302.0


##### DISCRETE VARIABLES ( Categories )

In [34]:
discrete_vars = [col for col in X_train.columns if X_train[col].nunique() <= 12]

In [35]:
discrete_vars

['ay', 'ceyrek', 'mevsim', 'mevsim_2', 'mevsim_3', 'otv', 'futbol', 'secim']

In [36]:
for col in discrete_vars:
    print(col)
    print(X_train[col].nunique())

ay
12
ceyrek
4
mevsim
4
mevsim_2
2
mevsim_3
3
otv
2
futbol
2
secim
2


In [37]:
for col in discrete_vars:
    print(col)
    print(X_holdout[col].nunique())

ay
12
ceyrek
4
mevsim
4
mevsim_2
2
mevsim_3
3
otv
2
futbol
1
secim
2


**Looks like in the test data we dont have 2 options for "futbol" column. Since we can not use different dimensionality data 
for train and test, we need to drop that column.**

**I will convert remaining columns datatypes to
category. Since they all have finite set of values. I will then one-hot encode these columns in order to apply it into the
machine learning algorithms since they only accept numbers.**

In [38]:
discrete_vars.remove("futbol")

In [39]:
discrete_vars

['ay', 'ceyrek', 'mevsim', 'mevsim_2', 'mevsim_3', 'otv', 'secim']

In [40]:
X_train[discrete_vars] = X_train[discrete_vars].astype("category")

In [41]:
X_holdout[discrete_vars] = X_holdout[discrete_vars].astype("category")

In [42]:
X_train.shape, X_holdout.shape

((75, 80), (12, 80))

In [43]:
X_train.drop("futbol", axis = 1, inplace = True)
X_holdout.drop("futbol", axis = 1, inplace = True)

In [44]:
X_train2 = pd.get_dummies(X_train, drop_first = True)

In [45]:
X_holdout2 = pd.get_dummies(X_holdout, drop_first = True)

In [46]:
X_train2.shape, X_holdout2.shape

((75, 94), (12, 94))

In [47]:
set(X_train2.columns) - set(X_holdout2.columns)

set()

## SCALING NUMERICAL FEATURES 

In [48]:
X_train2.head()

,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay,ay_2,ay_3,ay_4,ay_5,ay_6,ay_7,ay_8,ay_9,ay_10,ay_11,ay_12,ceyrek_2,ceyrek_3,ceyrek_4,mevsim_2,mevsim_3,mevsim_4,mevsim_2_2,mevsim_3_2,mevsim_3_3,otv_1,secim_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,571770.0,44,16,25,1,8,6,25,7.31,75.787368,2.352659,0.003470,1.772368,-0.008637,68.5,91.3,94.97,93.02,97.41,75.84,91.82,83.27,93.45,78.84,11.19,86.23,78.90,75.38,90.98,-19.7,-20.5,-6.5,-34.3,-19.2,-30.0,-20.1,1.4,-15.8,-17.6,-14.8,-20.3,-15.9,-31.1,11.3,4.9,86.9,87.2,98.7,85.1,85.6,79.4,93.7,102.7,48.5,79853.0,113.36,600658.0,588748.0,617001.0,602136.0,507079.0,470569.0,478926.0,543830.0,491472.0,526222.0,474710.0,528376.0,489523.0,471891.0,495093.0,517658.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2013-02-01,488879.0,46,17,32,1,9,7,33,7.03,76.660100,2.377090,0.010278,1.776035,0.002065,69.9,91.8,95.99,93.94,99.05,76.07,90.86,84.82,97.52,76.59,10.38,90.31,79.97,76.55,91.62,-19.5,-20.4,-5.6,-34.0,-18.9,-30.7,-21.7,0.4,-15.9,-18.1,-15.7,-16.6,-16.6,-31.1,7.4,7.8,87.7,87.7,99.3,87.1,85.8,78.7,92.3,105.4,48.5,79308.0,112.75,571770.0,600658.0,588748.0,587059.0,617001.0,507079.0,470569.0,559304.0,478926.0,491472.0,526222.0,539161.0,474710.0,489523.0,471891.0,524047.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2013-03-01,488910.0,43,19,29,1,8,7,28,7.29,74.911700,2.349043,-0.011940,1.810457,0.019013,84.3,92.4,97.36,93.62,98.05,75.42,90.79,83.14,97.78,81.24,11.90,88.30,79.83,74.83,85.94,-19.2,-20.2,-5.2,-32.4,-19.9,-30.9,-19.9,0.9,-17.3,-19.6,-18.6,-16.9,-17.8,-32.3,-2.6,9.0,87.7,87.7,99.8,85.6,85.2,81.6,93.9,108.7,48.5,77133.0,112.87,488879.0,571770.0,600658.0,553769.0,588748.0,617001.0,507079.0,562356.0,470569.0,478926.0,491472.0,535011.0,526222.0,474710.0,489523.0,525463.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2013-04-01,474507.0,48,21,28,1,8,8,27,6.13,75.620582,2.339733,-0.003979,1.799690,-0.005983,87.8,92.3,95.69,97.67,101.27,75.76,91.11,80.35,94.90,81.68,11.33,88.62,79.77,75.22,87.36,-18.6,-19.5,-5.9,-32.2,-19.8,-26.5,-19.5,3.9,-19.3,-21.5,-23.9,-13.6,-22.4,-30.9,-0.9,6.4,85.9,85.9,97.0,86.3,82.1,78.6,92.1,107.5,48.5,79236.0,113.85,488910.0,488879.0,571770.0,516520.0,600658.0,588748.0,617001.0,559328.0,507079.0,470569.0,478926.0,534727.0,491472.0,526222.0,474710.0,525412.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0
2013-05-01,468524.0,42,27,30,1,8,7,31,6.51,77.461800,2.370900,0.013146,1.826050,0.014435,97.9,92.8,99.34,101.12,104.74,75.50,94.43,82.75,95.95,85.71,14.79,89.36,79.94,77.17,85.41,-18.5,-19.3,-4.5,-32.8,-22.3,-26.4,-18.5,0.1,-16.6,-18.5,-16.0,-18.5,-23.3,-25.7,0.2,4.8,86.8,86.9,98.4,87.1,82.1,79.7,94.0,104.6,48.5,75980.0,112.25,474507.0,488910.0,488879.0,484099.0,571770.0,600658.0,588748.0,535579.0,617001.0,507079.0,470569.0,534236.0,478926.0,491472.0,526222.0,525395.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0


In [49]:
cols_to_normalize = [col for col in X_train2.columns if X_train2[col].dtype == "int64" or X_train2[col].dtype == "float64"]

In [50]:
scaler = MinMaxScaler()

In [51]:
scaler.fit(X_train2[cols_to_normalize])

MinMaxScaler()

In [52]:
X_train2[cols_to_normalize] = scaler.transform(X_train2[cols_to_normalize])

In [53]:
X_train2.head()

,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay,ay_2,ay_3,ay_4,ay_5,ay_6,ay_7,ay_8,ay_9,ay_10,ay_11,ay_12,ceyrek_2,ceyrek_3,ceyrek_4,mevsim_2,mevsim_3,mevsim_4,mevsim_2_2,mevsim_3_2,mevsim_3_3,otv_1,secim_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.637670,0.145833,0.000000,0.026316,0.0,0.00000,0.000000,0.000000,0.061748,0.873150,0.002535,0.406322,0.000000,0.312358,0.108557,0.284211,0.026846,0.708933,0.719512,0.638259,0.826715,0.748152,0.932298,0.780810,0.383966,0.694484,0.962658,0.866050,0.980452,0.000000,0.000000,0.000000,0.000000,0.151220,0.040724,0.068670,0.862745,0.133588,0.142336,0.354086,0.000000,0.289062,0.024145,0.549708,0.829876,0.034722,0.044674,0.095506,0.000000,0.113636,0.024096,0.065844,0.808777,0.446154,0.196205,0.990456,0.714968,0.674938,0.724735,0.761991,0.389213,0.277771,0.303280,0.649149,0.272216,0.341686,0.154291,0.608575,0.208179,0.144035,0.228442,0.557460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2013-02-01,0.415874,0.187500,0.012346,0.118421,0.0,0.01087,0.010638,0.106667,0.047096,0.914330,0.007327,0.432037,0.000796,0.353231,0.126437,0.297368,0.163758,0.731762,0.766024,0.659112,0.783394,0.836271,0.990308,0.716174,0.270042,0.890168,0.986441,0.923039,1.000000,0.011561,0.005650,0.092784,0.008357,0.165854,0.009050,0.000000,0.823529,0.129771,0.124088,0.319066,0.099196,0.261719,0.024145,0.435673,0.950207,0.062500,0.061856,0.129213,0.076046,0.120130,0.003012,0.008230,0.893417,0.446154,0.181306,0.978574,0.637670,0.707627,0.638496,0.710108,0.724735,0.389213,0.277771,0.715391,0.229887,0.215269,0.341686,0.663491,0.154291,0.208179,0.144035,0.597846,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2013-03-01,0.415957,0.125000,0.037037,0.078947,0.0,0.00000,0.010638,0.040000,0.060701,0.831830,0.001826,0.348114,0.008269,0.417962,0.310345,0.313158,0.347651,0.723821,0.737663,0.600181,0.780235,0.740762,0.994014,0.849756,0.483826,0.793765,0.983330,0.839260,0.826512,0.028902,0.016949,0.134021,0.052925,0.117073,0.000000,0.077253,0.843137,0.076336,0.069343,0.206226,0.091153,0.214844,0.000000,0.143275,1.000000,0.062500,0.061856,0.157303,0.019011,0.100649,0.090361,0.074074,0.996865,0.446154,0.121846,0.980912,0.415874,0.628339,0.674850,0.595549,0.638496,0.724735,0.389213,0.728457,0.201692,0.169628,0.215269,0.642360,0.341686,0.154291,0.208179,0.606797,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2013-04-01,0.377418,0.229167,0.061728,0.065789,0.0,0.00000,0.021277,0.026667,0.000000,0.865280,0.000000,0.378185,0.005932,0.322497,0.355045,0.310526,0.123490,0.824318,0.828985,0.631006,0.794675,0.582149,0.952965,0.862396,0.403657,0.809113,0.981996,0.858256,0.869884,0.063584,0.056497,0.061856,0.058496,0.121951,0.199095,0.094421,0.960784,0.000000,0.000000,0.000000,0.179625,0.035156,0.028169,0.192982,0.892116,0.000000,0.000000,0.000000,0.045627,0.000000,0.000000,0.000000,0.959248,0.446154,0.179338,1.000000,0.415957,0.400831,0.586673,0.467367,0.674850,0.638496,0.724735,0.715494,0.324873,0.139226,0.169628,0.640913,0.215269,0.341686,0.154291,0.606474,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0
2013-05-01,0.361409,0.104167,0.135802,0.092105,0.0,0.00000,0.010638,0.08000

In [54]:
X_holdout2.head()

,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay,ay_2,ay_3,ay_4,ay_5,ay_6,ay_7,ay_8,ay_9,ay_10,ay_11,ay_12,ceyrek_2,ceyrek_3,ceyrek_4,mevsim_2,mevsim_3,mevsim_4,mevsim_2_2,mevsim_3_2,mevsim_3_3,otv_1,secim_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-01,349634.0,59,45,73,38,31,41,65,19.50,64.453024,6.459143,0.044909,5.746105,0.051228,79.0,81.6,97.17,62.72,82.42,75.01,82.24,72.66,50.47,63.92,14.48,81.20,32.83,63.54,63.46,-6.5,-7.4,-0.7,-6.1,-11.1,-13.4,-11.1,-18.3,0.3,-1.1,-6.5,5.7,-4.1,4.5,5.1,-10.9,104.0,103.6,104.5,103.9,99.4,99.7,98.6,85.0,46.8,108311.0,72.73,400725.0,333456.0,342839.0,359007.0,465824.0,448594.0,382862.0,395717.0,379567.0,410789.0,456198.0,402317.0,459589.0,524049.0,457754.0,421854.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0
2019-05-01,392833.0,59,64,77,38,33,37,69,18.71,55.507248,6.779509,0.047255,6.060359,0.051854,86.2,81.1,94.46,59.91,70.15,69.14,73.95,69.02,39.79,53.93,10.30,76.68,32.11,55.28,56.29,-5.8,-6.5,-1.4,-3.7,-9.5,-12.1,-8.3,-23.7,0.6,-0.9,-4.3,5.3,-6.3,5.8,3.0,-11.6,105.0,105.0,104.9,105.1,104.2,101.6,93.9,79.7,45.3,111580.0,69.94,349634.0,400725.0,333456.0,361272.0,342839.0,465824.0,448594.0,390179.0,382862.0,379567.0,410789.0,390477.0,456198.0,459589.0,524049.0,412844.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1
2019-06-01,381932.0,62,100,94,29,33,27,82,15.72,57.742552,6.568718,-0.032090,5.822400,-0.040870,85.8,92.1,97.34,60.26,73.94,68.90,78.29,69.43,40.46,57.25,10.87,79.99,31.35,57.63,56.30,-6.2,-7.2,-3.1,-2.1,-9.3,-14.0,-10.5,-21.6,1.2,0.1,-3.1,6.3,-6.6,7.7,-7.5,-7.2,103.1,102.9,101.9,103.8,103.3,99.9,94.3,84.9,47.9,114985.0,72.32,392833.0,349634.0,400725.0,381064.0,333456.0,342839.0,465824.0,380885.0,448594.0,382862.0,379567.0,388482.0,410789.0,456198.0,459589.0,401909.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1
2019-07-01,346569.0,67,87,88,31,36,32,80,16.65,56.725613,6.380609,-0.029481,5.682395,-0.024638,84.7,81.0,96.88,59.80,73.37,67.54,76.97,65.82,40.26,59.39,11.04,79.86,34.47,56.49,55.58,-5.7,-6.6,-2.9,-4.9,-7.3,-11.8,-6.9,-22.8,0.6,-0.6,-6.9,3.0,-7.1,11.9,-11.7,-10.9,102.4,102.3,99.2,105.3,102.8,101.6,93.6,81.3,46.7,116480.0,75.01,381932.0,392833.0,349634.0,374800.0,400725.0,333456.0,342839.0,366903.0,465824.0,448594.0,382862.0,388744.0,379567.0,410789.0,456198.0,395438.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0
2019-08-01,328424.0,66,100,90,30,39,31,80,15.01,58.501034,6.267294,-0.018080,5.634906,-0.008428,77.4,80.7,94.88,61.68,74.27,69.76,77.81,69.59,38.72,60.84,12.47,79.19,36.29,58.25,58.27,-6.2,-7.1,-3.9,-6.2,-7.7,-12.9,-11.4,-22.7,1.6,0.6,-4.9,6.1,-7.6,12.9,-29.0,-5.4,102.7,102.6,99.6,107.8,103.1,100.5,91.9,87.1,48.0,117118.0,76.94,346569.0,381932.0,392833.0,373778.0,349634.0,400725.0,333456.0,367525.0,342839.0,465824.0,448594.0,384712.0,382862.0,379567.0,410789.0,386302.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0


In [55]:
scaler2 = MinMaxScaler()

In [56]:
scaler2.fit(X_holdout2[cols_to_normalize])

MinMaxScaler()

In [57]:
X_holdout2[cols_to_normalize] = scaler.transform(X_holdout2[cols_to_normalize])

In [58]:
X_holdout2.head()

,TV_Pazar,google_tv,google_bzd,google_cm,google_dolar,google_euro,google_doviz,google_vestel,Enflasyon,güven_endeksi,euro,euro_degisim,dolar,dolar_degisim,perakende_endeks_elektrikli,perakende_endeks_elektrikli_mev_tak,day_tuk_egilim,eko_durum,eko_durum_beklenti,mali_durum,maddi_durum_beklenti,maddi_durum,daya_tuk_uygunluk,tasarruf_uygunluk,otomobil_alma,tuketici_fiyat_beklenti,tuketici_fiyat_dusunce,guven_endeks,issiz_beklenti,European Union_CC,Euro area_CC,Germany_CC,Spain_CC,France_CC,Italy_CC,United Kingdom_CC,Turkey_CC,European Union_RC,Euro area_RC,Germany_RC,Spain_RC,France_RC,Italy_RC,United Kingdom_RC,Turkey_RC,European Union_ES,Euro area_ES,Germany_ES,Spain_ES,France_ES,Italy_ES,United Kingdom_ES,Turkey_ES,pmi,gercek_yp_mevduat,reel_döviz,lag1,lag2,lag3,mean_son_3_ay,lag4,lag5,lag6,mean_son_6_ay,lag7,lag8,lag9,mean_son_9_ay,lag10,lag11,lag12,mean_son_12_ay,ay_2,ay_3,ay_4,ay_5,ay_6,ay_7,ay_8,ay_9,ay_10,ay_11,ay_12,ceyrek_2,ceyrek_3,ceyrek_4,mevsim_2,mevsim_3,mevsim_4,mevsim_2_2,mevsim_3_2,mevsim_3_3,otv_1,secim_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-01,0.043288,0.458333,0.358025,0.657895,0.373737,0.250000,0.372340,0.533333,0.699634,0.338329,0.807992,0.562851,0.862736,0.541001,0.242656,0.028947,0.322148,-0.042928,0.294385,0.563010,0.394404,0.144969,0.319698,0.352198,0.846695,0.453237,-0.061347,0.289333,0.139890,0.763006,0.740113,0.597938,0.785515,0.546341,0.791855,0.454936,0.090196,0.748092,0.744526,0.677043,0.697051,0.750000,0.740443,0.368421,0.174274,0.628472,0.608247,0.421348,0.714829,0.561688,0.635542,0.267490,0.253918,0.315385,0.974193,0.199065,0.179996,-0.025753,-0.112107,-0.074671,0.263288,0.210695,0.010058,0.015099,-0.105340,-0.078248,0.086946,-0.033306,0.099282,0.333781,0.092606,-0.048136,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0
2019-05-01,0.158878,0.458333,0.592593,0.710526,0.373737,0.271739,0.329787,0.586667,0.658294,-0.083784,0.870829,0.571714,0.930963,0.543392,0.334610,0.015789,-0.041611,-0.112655,-0.053602,0.030825,0.020307,-0.061967,0.167474,0.065211,0.258790,0.236451,-0.077351,-0.113005,-0.079108,0.803468,0.790960,0.525773,0.852368,0.624390,0.850679,0.575107,-0.121569,0.759542,0.751825,0.762646,0.686327,0.664062,0.766600,0.307018,0.145228,0.663194,0.656357,0.443820,0.760456,0.717532,0.692771,0.074074,0.087774,0.200000,1.063561,0.144721,0.043288,0.158878,-0.140748,-0.066877,-0.112107,0.263288,0.210695,-0.008609,-0.094223,-0.191830,-0.078248,-0.093594,0.086946,0.099282,0.333781,-0.105090,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1
2019-06-01,0.129710,0.520833,1.037037,0.934211,0.282828,0.271739,0.223404,0.760000,0.501832,0.021690,0.829484,0.271998,0.879300,0.189253,0.329502,0.305263,0.344966,-0.103970,0.053885,0.009066,0.216155,-0.038658,0.177024,0.160586,0.338959,0.395204,-0.094243,0.001461,-0.078803,0.780347,0.751412,0.350515,0.896936,0.634146,0.764706,0.480687,-0.039216,0.782443,0.788321,0.809339,0.713137,0.652344,0.804829,0.000000,0.327801,0.597222,0.584192,0.275281,0.711027,0.688312,0.641566,0.090535,0.250784,0.400000,1.156647,0.191079,0.158878,0.018650,0.064582,0.001232,-0.140748,-0.112107,0.263288,-0.048395,0.127550,-0.179843,-0.191830,-0.103753,-0.078248,0.086946,0.099282,-0.174212,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1
2019-07-01,0.035087,0.625000,0.876543,0.855263,0.303030,0.304348,0.276596,0.733333,0.550497,-0.026295,0.792588,0.281853,0.848904,0.251245,0.315453,0.013158,0.283221,-0.115385,0.037720,-0.114234,0.156588,-0.243889,0.174173,0.222063,0.362869,0.388969,-0.024894,-0.054067,-0.100794,0.809249,0.785311,0.371134,0.818942,0.731707,0.864253,0.635193,-0.086275,0.759542,0.762774,0.661479,0.624665,0.632812,0.889336,-0.122807,0.174274,0.572917,0.563574,0.123596,0.768061,0.672078,0.692771,0.061728,0.137931,0.307692,1.197518,0.243475,0.129710,0.137217,-0.091366,-0.020324,0.064582,-0.140748,-0.112107,-0.108250,0.185683,0.059283,-0.179843,-0.102419,-0.191830,-0.078248,0.086946,-0.215116,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,

In [59]:
X_train2.shape, X_holdout2.shape

((75, 94), (12, 94))

In [60]:
y.shape

(87,)

In [61]:
X_train2.to_csv("Training_Data.csv")
X_holdout2.to_csv("Testing_Data.csv")
y.to_csv("Target.csv")